<a href="https://colab.research.google.com/github/rk119/F20BC/blob/main/Copy_of_F20BC_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biologically-Inspired Computation (F20BC) Coursework

**Overview:**

The following coursework aims to build an Artificial Neural Network (ANN) with Feedforward and instead of Backpropogation, the neural network is trained using the Particle Swarm Optimisation (PSO) algorithm, two biologically-inspired techniques which are taught in this course. The ANN structure and PSO algorithms are implemented from scratch, to build a deeper understanding as well as investigate or experiment with them flexibly to get a comprehensive idea on how the ANN is optimised using PSO in low level for a specified task.



# Data preprocessing

In [642]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Imports

In [643]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

## Variables

In [644]:
skipHeader = False
normalize = True
test_split = 0.2
input_nodes = 4

### Load the data

In [645]:
if skipHeader:
    df = pd.read_csv("/content/drive/MyDrive/F20BC-Dataset/data_banknote_authentication.csv", skiprows=1)
else:
    df = pd.read_csv("/content/drive/MyDrive/F20BC-Dataset/data_banknote_authentication.csv", header=None)

### View the first 5 instances of the data

In [646]:
df.head()

,0,1,2,3,4
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


## Viewing the number of rows and columns

In [647]:
df.shape

(1372, 5)

## Split the data into features and target

**Note:** For one dataset, the target class is seperated into a different variable since we do not want the model to consider it as an input feature

In [648]:
def split_input_output(data):
    X = data.iloc[:, :-1].to_numpy()
    Y = data.iloc[:, -1].to_numpy()

    return [X, Y]

In [649]:
x = split_input_output(df)[0]
y = split_input_output(df)[1]

In [650]:
x.shape

(1372, 4)

In [651]:
x

array([[  3.6216 ,   8.6661 ,  -2.8073 ,  -0.44699],
       [  4.5459 ,   8.1674 ,  -2.4586 ,  -1.4621 ],
       [  3.866  ,  -2.6383 ,   1.9242 ,   0.10645],
       ...,
       [ -3.7503 , -13.4586 ,  17.5932 ,  -2.7771 ],
       [ -3.5637 ,  -8.3827 ,  12.393  ,  -1.2823 ],
       [ -2.5419 ,  -0.65804,   2.6842 ,   1.1952 ]])

In [652]:
y

array([0, 0, 0, ..., 1, 1, 1])

## Normalize if specified

In [653]:
def norm(data):
    scaler = MinMaxScaler()
    return scaler.fit_transform(data)

In [654]:
x_norm = norm(x)
x_norm

array([[0.76900389, 0.83964273, 0.10678269, 0.73662766],
       [0.83565902, 0.82098209, 0.12180412, 0.64432563],
       [0.78662859, 0.41664827, 0.31060805, 0.78695091],
       ...,
       [0.23738543, 0.01176814, 0.98560321, 0.52475518],
       [0.25084193, 0.20170105, 0.76158701, 0.6606745 ],
       [0.32452819, 0.49074676, 0.34334762, 0.88594888]])

In [655]:
x_norm.shape

(1372, 4)

In [656]:
len(x_norm)

1372

In [657]:
len(y)

1372

## Split data into Train and Test

In [658]:
x_norm, x_test, y, y_test = train_test_split(x_norm, y, test_size=test_split, random_state=42)

# Artificial Neural Network (ANN)

Artificial Neural Networks (ANNs) are computational models inspired by the human brain. They are composed of interconnected units or nodes called artificial neurons, which simulate the way biological neurons signal each other. ANNs are used in machine learning for pattern recognition and data classification, among other tasks.

In an ANN, neurons are organized in layers: an input layer to receive signals, one or more hidden layers to process them, and an output layer to deliver the final result. The network learns by adjusting the weights of connections based on the data it processes, typically through a method called backpropagation combined with a gradient descent optimization algorithm.

## Functions to utilize later

### Activation Functions

In [659]:
class ActivationFunctions:
  def evaluate(self,x):
    pass
  def derivate(self,x):
    pass

class Identity:
  def evaluate(self,x):
    return x
  def derivative(self,x):
    return 1

class Sigmoid(ActivationFunctions):
  def evaluate(self,x):
    x_clipped = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x_clipped))
  def derivative(self,x):
    f = self.evaluate(x)
    return f * (1-f)

class Tanh(ActivationFunctions):
  def evaluate(self,x):
    return np.tanh(x)
  def derivative(self,x):
    f = self.evaluate(x)
    return 1 - f ** 2

class relu(ActivationFunctions):
  def evaluate(self,x):
    return np.maximum(0, x)
  def derivative(self,x):
    return (x > 0).astype(float)

### Loss Functions

In [660]:
class LossFunctions:
  def evaluate(self,x):
    pass
  def derivate(self,x):
    pass

# y is predictions made by the neural network
# t is target/actual numbers corresponding to inputs
class Mse(LossFunctions):
  def evaluate(self, y, t):
    return ((t - y) ** 2).mean()
  def derivative(self, y, t):
    return 2 * (y - t) / len(y)

class BinaryCrossEntropy(LossFunctions):
  def evaluate(self, y, t):
    y_pred = np.clip(y, 1e-7, 1 - 1e-7)
    term0 = (1 - t) * np.log(1 - y_pred + 1e-7)
    term1 = t * np.log(y_pred + 1e-7)
    return - (term0 + term1).mean()

  def derivative(self, y, t):
    y_pred = np.clip(y, 1e-7, 1 - 1e-7)
    return (t / y_pred) - (1 - t) / (1 - y_pred)

class Hinge(LossFunctions):
  def evaluate(self, y, t):
    return np.maximum(0, 1 - t * y).mean()

  def derivative(self, y, t):
    return np.where(t * y < 1, -t, 0)

### Neural Network

In [661]:
class InputLayer:
    def __init__(self, input_size):
        self.nb_nodes = input_size

    def forward(self, input_data):
        return input_data

class Layer:
    def __init__(self, input_size, nodes, activation, weights=None, biases=None):
        self.nb_nodes = nodes
        self.W = weights if weights is not None else np.random.randn(input_size, nodes)
        self.B = biases if biases is not None else np.random.randn(nodes)
        self.activation = activation

    def forward(self, input_data):
        self.X_in = input_data
        z = np.dot(input_data, self.W) + self.B
        out = self.activation.evaluate(z)
        return out

class NeuralNetwork:
    def __init__(self, configuration, position=None):
        self.layers = []
        input_size = configuration[0]

        # The input layer is simply added as a pass-through layer
        self.layers.append(InputLayer(input_size))

        # If a position vector is provided, it contains weights and biases for each layer
        if position is not None:
            for idx, (nodes, activation) in enumerate(configuration[1:]):
                weights, biases = position[idx]
                # print("layer ", idx, "W = ",weights, "B = ", biases )
                layer = Layer(input_size, nodes, activation, weights=weights, biases=biases)
                self.add(layer)
                input_size = nodes  # Update input size for the next layer
        else:
            # If no position vector, initialize layers with random weights and biases
            for nodes, activation in configuration[1:]:
                layer = Layer(input_size, nodes, activation)
                self.add(layer)
                input_size = nodes  # Update input size for the next layer

    def add(self, layer):
        self.layers.append(layer)

    def forward(self, input_data):
        for layer in self.layers:
            input_data = layer.forward(input_data)
        return input_data

    def print_layers(self):
        for i, layer in enumerate(self.layers):
            if isinstance(layer, InputLayer):
                print(f"Layer {i}: Input Layer with {layer.nb_nodes} nodes")
            elif isinstance(layer, Layer):
                print(f"Layer {i}: Hidden Layer with {layer.nb_nodes} nodes, Activation Function: {layer.activation.__class__.__name__}")
                print(f"  Weights Shape: {layer.W.shape}, Biases Shape: {layer.B.shape}")
                print(f"  Weights= : {layer.W}, Biases= {layer.B}")
            else:
                print(f"Layer {i}: Unknown Layer Type")

    def flatten_weights_and_biases(self):
        flattened_vector = []
        for layer in self.layers:
            if isinstance(layer, Layer):
                # Flatten and append weights and biases of this layer
                flattened_weights = layer.W.flatten()
                flattened_biases = layer.B.flatten()
                flattened_vector.extend(flattened_weights.tolist())
                flattened_vector.extend(flattened_biases.tolist())
        return flattened_vector

### Reconstructing Weights and Biases from Flattened Vector

In [662]:
def unflatten_weights_and_biases(flattened_vector, configuration):
    position = []
    index = 0

    # Skip the input layer configuration, as it doesn't have weights or biases
    for nodes, _ in configuration[1:]:
        # Previous layer's node count is the number of rows for weights
        input_size = configuration[0] if not position else position[-1][0].shape[1]

        # Calculate the number of weights and biases
        num_weights = input_size * nodes
        num_biases = nodes

        # Extract weights and biases from the flattened vector
        weights = flattened_vector[index : index + num_weights]
        biases = flattened_vector[index + num_weights : index + num_weights + num_biases]

        # Reshape weights to the correct dimensions and add to position
        position.append((np.array(weights).reshape(input_size, nodes), np.array(biases)))

        # Update the index
        index += num_weights + num_biases

    return position

In [663]:
# Creating objects

relu_activation = relu()
sigmoid_activation = Sigmoid()
tanh_activation = Tanh()

In [664]:
# Configuring the layers with number of nodes, activation and the total number of elements here are the number of layers
configuration = [
    x_norm.shape[1],
    [3, relu_activation],
    [4, relu_activation],
    [1, sigmoid_activation]
]

loss_function = BinaryCrossEntropy()

### Fitness Function

In [665]:
def fitness(X, Y, W):
    # Rebuild the neural network with the given weights
    position = unflatten_weights_and_biases(W, config)
    # print(position)
    neural_network = NeuralNetwork(config, position=position)

    # Perform forward pass
    predictions = neural_network.forward(X)

    # Calculate loss
    loss = loss_function.evaluate(predictions.flatten(), Y)
    return loss

### Function to Calculate Accuracy

In [666]:
def get_accuracy(Y, Y_pred):
    """
    Calcualtes accuracy.
    :param Y: int(N, )
        Correct labels.
    :param Y_pred: int(N, ) | double(N, C)
        Predicted labels of shape(N, ) or (N, C) in case of one-hot vector.
    :return: double
        Accuracy.
    """
    predicted_classes = (Y_pred >= 0.5).astype(int) # Convert probabilities to class labels
    return (Y == predicted_classes).mean()

## Attempting the functions

In [667]:
nn = NeuralNetwork(configuration)

In [668]:
nn.print_layers()

Layer 0: Input Layer with 4 nodes
Layer 1: Hidden Layer with 3 nodes, Activation Function: relu
  Weights Shape: (4, 3), Biases Shape: (3,)
  Weights= : [[ 0.70612769  0.58893751  1.09583642]
 [-1.92999508  1.18800863  1.00200309]
 [-0.10345042 -0.05041144 -1.68295593]
 [-0.46027475  0.43858819 -0.94545983]], Biases= [ 2.97814159 -0.49802143 -0.02031129]
Layer 2: Hidden Layer with 4 nodes, Activation Function: relu
  Weights Shape: (3, 4), Biases Shape: (4,)
  Weights= : [[ 0.74317602 -0.20731627  0.15664229  0.92730152]
 [-1.088695   -0.49203727 -0.52426945  0.88569827]
 [-0.96172267 -1.514597    0.09332996 -1.03053707]], Biases= [0.10428879 0.33431689 2.0444348  0.63536073]
Layer 3: Hidden Layer with 1 nodes, Activation Function: Sigmoid
  Weights Shape: (4, 1), Biases Shape: (1,)
  Weights= : [[ 0.34321589]
 [-0.63036775]
 [ 0.927578  ]
 [-0.86975417]], Biases= [-0.19195896]


In [669]:
flattened_vector = nn.flatten_weights_and_biases()
print(flattened_vector)

[0.7061276920980305, 0.5889375057107044, 1.0958364153486024, -1.929995082347134, 1.1880086290101022, 1.0020030865892204, -0.10345042047052429, -0.05041143713699172, -1.6829559323684258, -0.46027474809017105, 0.4385881866028287, -0.945459831889112, 2.9781415926196546, -0.4980214334897505, -0.02031128680650146, 0.7431760201117457, -0.2073162710128619, 0.15664228525696874, 0.927301519618885, -1.08869499901303, -0.4920372650224045, -0.5242694460474194, 0.8856982668586484, -0.9617226729130631, -1.5145970007251255, 0.09332995881275337, -1.0305370733568575, 0.10428878585073756, 0.33431689066577785, 2.0444348017343392, 0.6353607271085187, 0.34321589367016303, -0.6303677524392395, 0.9275779972462859, -0.8697541695345179, -0.19195896174597402]


In [670]:
input_data = x_norm

nn_init_pred = nn.forward(input_data)

In [671]:
len(nn_init_pred)

1097

# Particle Swarm Optimization (PSO)

It is a computational method that optimizes a problem by iteratively trying to improve a candidate solution with regard to a given measure of quality. It solves a problem by having a population of candidate solutions, known as particles, and moving these particles around in the search-space according to simple mathematical formulae over the particle's position and velocity. Each particle's movement is influenced by its local best known position but is also guided toward the best known positions in the search-space, which are updated as better positions are found by other particles. This is expected to move the swarm toward the best solutions. PSO is widely used in optimization problems where the solution space is vast and multidimensional.

The Particle class represents a single solution in the optimization process. Each particle has its own position and velocity in the solution space.

In [672]:
class Particle:
    """
    Particle is a neural network representing a potential solution.
    """
    def __init__(self, num_dim, x_range, v_range, r1_range, r2_range, r3_range):
        """
        Particle class constructor
        :param num_dim: Number of dimensions.
        :param x_range: Range of dimension.
        :param v_range: Range of velocity.
        """

        self.x = np.random.uniform(x_range[0], x_range[1], (num_dim, )) # particle position
        self.v = np.random.uniform(v_range[0], v_range[1], (num_dim, )) # particle velocity
        self.pbest = np.inf                                             # personal best fitness
        self.pbestpos = np.zeros((num_dim, ))                           # personal best position
        self.informants_best_position = np.zeros((num_dim, ))           # informants best position
        self.informants = []                                            # particle's informants
        self.r1_range = r1_range                                        # array with min and max range for r1
        self.r2_range = r2_range                                        # array with min and max range for r2
        self.r3_range = r3_range                                        # array with min and max range for r3

    def update_informant_best(self, swarm):
        best_fitness = np.inf
        for informant in self.informants:
            if swarm[informant].pbest < best_fitness:
                best_fitness = swarm[informant].pbest
                self.informants_best_position = swarm[informant].pbestpos.copy()

    def update_velocity(self, global_best_position, alpha, beta, gamma, delta, r1_range, r2_range, r3_range):
        r1 = np.random.uniform(self.r1_range[0], self.r1_range[1])
        r2 = np.random.uniform(self.r2_range[0], self.r3_range[1])
        r3 = np.random.uniform(self.r3_range[0], self.r3_range[1])
        # r1, r2, r3 = np.random.rand(3)  # Random coefficients for stochastic components
        cognitive_component = beta * r1 * (self.pbestpos - self.x)
        social_component = gamma * r2 * (self.informants_best_position - self.x)
        global_component = delta * r3 * (global_best_position - self.x)
        self.v = alpha * self.v + cognitive_component + social_component + global_component

This class represents a group of particles, working together to explore and optimize a problem space. It initializes with a set number of particles, each having its dimensions, velocities, and inertia weights. Particles communicate amongst each other through informants, allowing them to learn from each other's experiences. The optimize method iteratively updates each particle's position and velocity based on personal and collective experiences, steering the swarm towards the best solution.

In [673]:
class Swarm:
    """
    The Swarm class is a collection of potential solutions, each represented by a particle.
    """
    def __init__(self, no_particle, num_dim, x_range, v_range, iw_range, c, num_informants, r1_range, r2_range, r3_range):
        """
        Swarm class constructor.
        :param no_particle:  Number of particles
        :param num_dim: Number of dimensions.
        :param x_range: Range of dimensions.
        :param v_range: Range of velocities.
        :param iw_range: Range of interia weights.
        :param c: c[0] -> cognitive parameter, c[1] -> social parameter, c[2] -> Global weight, c[3] -> Inertia weight
        :param num_informants: Number of informants
        """
        self.p = np.array([Particle(num_dim, x_range, v_range, r1_range, r2_range, r3_range) for i in range(no_particle)])
        self.gbest = np.inf
        self.gbestpos = np.zeros((num_dim, ))
        self.x_range = x_range
        self.v_range = v_range
        self.iw_range = iw_range
        self.c0 = c[0]            # Cognitive weight
        self.c1 = c[1]            # Social weight
        self.c2 = c[2]            # Global weight
        self.c3 = c[3]            # Inertia weight
        self.num_dim = num_dim    # Number of dimensions, in this case the total number of weights & biases
        self.num_informants = num_informants
        self.assign_informants()

    def print_informants(self):
        """
        Function prints the informants of every particle.
        """
        for i, particle in enumerate(self.p):
            # Print the particle's index and its informants
            print(f"Particle {i} informants: {particle.informants}")

    def assign_informants(self):
        """
        Function assigns informants to every particle
        """
        for i, particle in enumerate(self.p):
            informants = set()
            # add informants until they are num_informants in informants set
            while len(informants) < self.num_informants:
                # Randomly select a potential informant
                possible_informant = np.random.randint(0, len(self.p))
                # if selected informant is not the particle itself, add it to set
                if possible_informant != i:
                    informants.add(possible_informant)
            particle.informants = np.array(list(informants))

    def optimize(self, function, X, Y,  print_step,  iter):
        """
        Function used to start optimization.
        :param function: function
            Function to be optimized
        :param X: input data
            Used in forward pass.
        :param Y: target class
            Used to calculate loss.
        :param print_step: int
            Step for printing
        :param iter: int
            Number of iterations.
        """
        for i in range(iter):
            for particle in self.p:
                # print("particle.x",particle.x)
                fitness = function(X, Y, particle.x)
                # print("fitness",fitness)
                # print("particle.pbest",particle.pbest)

                if fitness < particle.pbest:
                    particle.pbest = fitness
                    particle.pbestpos = particle.x.copy()

                if fitness < self.gbest:
                    self.gbest = fitness
                    self.gbestpos = particle.x.copy()

            for particle in self.p:
                # update informants best
                particle.update_informant_best(self.p)
                # update particle velocity
                particle.update_velocity(self.gbestpos, self.c3, self.c0, self.c1, self.c2, r1_range, r2_range, r3_range)
                # update particle position
                particle.x = particle.x +  particle.v

            if i % print_step == 0:
                print('iteration#: ', i+1,  ' loss: ', fitness)

        print("global best loss: ", self.gbest)

    def best_solution(self):
        '''
        return: array of parameters/weights.
        '''
        return self.gbestpos

In [674]:
def num_dim(configuration):
  total_weights = 0
  total_biases = 0

  # The number of nodes in the previous layer, initially the input layer
  prev_nodes = configuration[0]

  # Loop through each layer (excluding the input layer)
  for layer in configuration[1:]:
      # Extract the number of nodes in the current layer
      nodes = layer[0]

      # Calculate weights and biases for the current layer
      weights = prev_nodes * nodes
      biases = nodes

      # Add to total weights and biases
      total_weights += weights
      total_biases += biases

      # Update prev_nodes for the next iteration
      prev_nodes = nodes

  # Output the total number of dimensions
  total_dimensions = total_weights + total_biases
  return total_dimensions

### Running the ANN model with manual parameters

In [679]:
def ann(config, X_train,  Y_train, no_solution = 20,  w_range = (0.0, 1.0), lr_range = (0.0, 1.0), iw_range = (0.0, 1.0), c = (0.5, 0.3, 0.2, 0.9), num_informants = 10, loss_function = BinaryCrossEntropy(), iterations=100, r1_range = [0,1], r2_range = [0,1], r3_range = [0,1]):
    # c[0] -> cognitive factor, c[1] -> global factor,  c[2] -> social factor, c[3] -> inertial weight
    no_dim = num_dim(config)

    s = Swarm(no_solution, no_dim, w_range, lr_range, iw_range, c, num_informants, r1_range, r2_range, r3_range)
    s.print_informants()

    # Drop column/s if user picks number of input nodes < number of features
    # Pick number of input nodes between 1 and num of features
    num_features = X_train.shape[1]  # number of features
    number_of_input_nodes = config[0]

    if (number_of_input_nodes < num_features):
      X_train = X_train.iloc[:, :number_of_input_nodes]


    print(X_train)

    # Train:Test Split
    test_size = 0.2  # User defined split ratio
    X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=test_size, random_state=42)


    s.optimize(fitness, X_train, Y_train, no_solution, iterations)
    W = s.best_solution()

    # Perform forward pass with the best solution (weights)
    best_position = unflatten_weights_and_biases(W, config)
    best_nn = NeuralNetwork(config, position=best_position)

    Y_pred = best_nn.forward(X_train).flatten()
    Y_pred2 = best_nn.forward(X_test).flatten()

    # Calculate and print accuracy
    train_accuracy = get_accuracy(Y_train.flatten(), Y_pred)
    # # Calculate and print accuracy
    test_accuracy = get_accuracy(Y_test.flatten(), Y_pred2)

    return [train_accuracy, test_accuracy]

In [680]:
accuracies = ann(configuration, x_norm, y)
print("Train Accuracy: %.3f" % accuracies[0])
print("Test Accuracy: %.3f" % accuracies[1])

Particle 0 informants: [ 1  2  4  6 11 13 14 15 16 17]
Particle 1 informants: [ 0  6  7  8 10 11 12 17 18 19]
Particle 2 informants: [ 0  4  5  7  9 11 12 13 15 18]
Particle 3 informants: [ 1  5  6  9 12 13 14 16 17 19]
Particle 4 informants: [ 1  8  9 10 11 12 13 14 15 19]
Particle 5 informants: [ 1  2  3  6  7 14 15 16 18 19]
Particle 6 informants: [ 0  4  5  7 11 13 14 15 17 18]
Particle 7 informants: [ 1  4  5  8  9 12 13 16 17 19]
Particle 8 informants: [ 5  7  9 10 13 14 15 16 18 19]
Particle 9 informants: [ 1  2  4  5  6  8 10 12 13 18]
Particle 10 informants: [ 3  5  6  7  8  9 13 14 16 19]
Particle 11 informants: [ 2  6  9 10 12 13 14 17 18 19]
Particle 12 informants: [ 0  2  4  5  6  8  9 14 16 17]
Particle 13 informants: [ 0  3  4  5  6  7 10 17 18 19]
Particle 14 informants: [ 5  6  9 10 11 15 16 17 18 19]
Particle 15 informants: [ 0  2  4  5  7  8 13 14 17 18]
Particle 16 informants: [ 0  4  7  8  9 12 13 14 15 17]
Particle 17 informants: [ 0  1  4  6  7  9 10 12 13 19]
Pa

ValueError: ignored

# Experimenting

In [ ]:
import itertools

def generate_configurations(min_layers, max_layers, min_nodes, max_nodes):
    configurations = []
    for num_layers in range(min_layers, max_layers + 1):
        for nodes_per_layer in itertools.product(range(min_nodes, max_nodes + 1), repeat=num_layers):
            configurations.append(nodes_per_layer)
    return configurations

# Example user inputs
min_layers = 1  # User-defined minimum number of hidden layers
max_layers = 3  # User-defined maximum number of hidden layers
min_nodes = 3   # User-defined minimum number of nodes per layer
max_nodes = 5   # User-defined maximum number of nodes per layer

# Generate configurations
configurations = generate_configurations(min_layers, max_layers, min_nodes, max_nodes)

results = []
for config in configurations:
    # Assuming all hidden layers use relu_activation for simplicity
    full_config = [input_nodes] + [[nodes, relu_activation] for nodes in config] + [[1, sigmoid_activation]]
    accuracies = ann(full_config, x_norm, y, x_test, y_test)
    train_acc = accuracies[0]
    test_acc = accuracies[1]
    results.append((config, train_acc, test_acc))

# Find the best configuration
best_config = max(results, key=lambda x: x[2])  # Select based on test accuracy

print(f"Best Configuration: {best_config[0]}")
print(f"Train Accuracy: {best_config[1]}")
print(f"Test Accuracy: {best_config[2]}")

# Plotting results
train_accuracies = [result[1] for result in results]
test_accuracies = [result[2] for result in results]
labels = [str(config[0]) for config in results]

plt.figure(figsize=(12, 6))
plt.plot(labels, train_accuracies, label='Train Accuracy')
plt.plot(labels, test_accuracies, label='Test Accuracy')
plt.xlabel('Configurations')
plt.ylabel('Accuracy')
plt.title('ANN Configurations vs. Accuracy')
plt.legend()
plt.show()